# Ngram Text Wordcloud

The following notebook takes you through the code to create a ngram (unigram, bigram, trigram, et cetera) wordcloud.

###  Before we begin
Before we start, you will need to have set up a [Carbonate account](https://kb.iu.edu/d/aolp) in order to access [Research Desktop (ReD)](https://kb.iu.edu/d/apum). You will also need to have access to ReD through the [thinlinc client](https://kb.iu.edu/d/aput). If you have not done any of this, or have only done some of this, but not all, you should go to our [textPrep-Py.ipynb](https://github.com/cyberdh/Text-Analysis/blob/master/Intro/Python/Py_notebooks/textPrep-Py.ipynb) before you proceed further. The textPrep-Py notebook provides information and resources on how to get a Carbonate account, how to set up ReD, and how to get started using the Jupyter Notebook on ReD.   

### Run CyberDH environment
The code in the cell below points to a Python environment specificaly for use with the Python Jupyter Notebooks created by Cyberinfrastructure for Digital Humanities. It allows for the use of the different packages in our notebooks and their subsequent data sets.

##### Packages
- **sys:** Provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter. It is always available.
- **os:** Provides a portable way of using operating system dependent functionality.

#### NOTE: This cell is only for use with Research Desktop. You will get an error if you try to run this cell on your personal device!!

In [1]:
import sys
import os
sys.path.insert(0,"/N/u/cyberdh/Carbonate/dhPyEnviron/lib/python3.6/site-packages")
os.environ["NLTK_DATA"] = "/N/u/cyberdh/Carbonate/dhPyEnviron/nltk_data"

### Include necessary packages for notebook 

Python's extensibility comes in large part from packages. Packages are groups of functions, data, and algorithms that allow users to easily carry out processes without recreating the wheel. Some packages are included in the basic installation of Python, others created by Python users are available for download.

In your terminal, packages can be installed by simply typing `pip install nameofpackage --user`. However, since you are using ReD and our Python environment, you will not need to install any of the packages below to use this notebook. Anytime you need to make use of a package, however, you need to import it so that Python knows to look in these packages for any functions or commands you use. Below is a brief description of the packages we are using in this notebook:  

- **textblob:** Library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more. 

- **nltk:** Platform for building Python programs to work with human language data.

- **re:** Provides regular expression matching operations similar to those found in Perl.

- **string:** contains a number of useful constants and classes, as well as some deprecated legacy functions that are also available as methods on strings.

- **pandas:** An open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

- **collections:** Implements specialized container datatypes providing alternatives to Python's general purpose built-in containers: dict, list, set, and tuple.

- **wordcloud:** A simple wordcloud generator in Python.

- **PIL:** Stands for Python Imaging Library and adds image processing capabilities.

- **numpy:** The fundamental package for scientific computing with Python.

- **operator:** Exports a set of functions corresponding to the intrinsic operators of Python.

- **glob:** Finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order.

- **matplotlib:** A Python 2D plotting library which produces publication quality figures in a variety of hardcopy formats and interactive environments across platforms.

In [2]:
from textblob import TextBlob
from nltk.corpus import stopwords
import nltk
import re
import string
import pandas as pd
from collections import Counter, defaultdict
import wordcloud
from wordcloud import STOPWORDS
from PIL import Image
import numpy as np
import operator
import glob
import matplotlib.pyplot as plt

#%matplotlib inline

#### File paths
Here we are saving as variables different file paths that we need in our code. We do this so that they are easier to call later and so that you can make most of your changes now and not need to make as many changes later. 

First we use the `os` package above to find our `['HOME']` directory using the `environ` function. This will work for any operating system, so if you decide to try this out on your personal computer instead of ReD, the `homePath` variable will still be the path to your 'home' directory, so no changes are needed.

Next, we combine the `homePath` variable with the folder names that lead to where our data is stored. Note that we do not use any file names yet, just the path to the folder. This is because we have the option later to read in a single file or an entire directory. You will want to change the folder names to match your folder names in your file path.

Now we add the `homePath` variable to other folder names that lead to a folder where we will want to save any output generated by this code. You again will want to change the file names in the appropriate cells down below to match your own file names. We save this file path as the variable `dataResults`.

In [3]:
homePath = os.environ["HOME"]
dataHome = os.path.join(homePath, "Text-Analysis-master", "data", "shakespeareDated")
dataResults = os.path.join(homePath, "Text-Analysis-master", "Output")

#### Set needed variables

Now we are setting needed variables that will help determine what the code will do farther down. We again do this so that they are easier to call later and so that you can make most of your changes now and not need to make as many changes later.

The first variable `data` is set as being equal to `"*.txt"`. This is where you determine if you want to read in an entire directory of '.txt' files or a single '.txt' file. If you want the entire directory, leave `data` equal to `"*.txt"`. If you want a single document then replace the "\*" with the name of the file, so `data = "*.txt"` becomes `data = "myFileName.txt"`. 

If you want to use the stopword list that comes with the nltk package then set `nltkStop` equal to **True**. If you do not wish to use the nltk stopword list then set `nltkStop` equal to **False**.

If you have created your own custom stopword list and wish to use that, then set `customStop` equal to **True**. If you do not have your own custom stopword list then set `customStop` equal to **False**.

**NOTE: You can use both the nltk and custom stopword lists or you can use neither or just one or the other. You do NOT need to set them both to True or both to False. Use whatever works best for you.**

The `ng` variable (short for ngram) is where you determine if you want a unigram (basically a word frequency count), a bigram (most common 2 word pairs), or a trigram (most common three word phrases). This will be used in the code further down.

Now we choose the language we will be using for the nltk stopwords list. If you need a different language, simply change `'english'` in the `stopLang` variable to the anglicized name of the language you wish to use (e.g. 'spanish' instead of 'espanol' or 'german' instead of 'deutsch'). If you need to see the list of available languages in nltk simply remove the `#` from in front of `#print(" ".join(stopwords.fileids()))` on the last line and run the cell. A list of available languages will display below the cell.

The `stopWords` variable is simply an empty list (that is what the square brackets [] indicate). This is where the words from the nltk stopword list or your custom stopword list or both combined or neither (depending on what you decide) will reside later on. You do not need to do anything to this line of code.

The `cleanText` variable is another empty list. This is where your document(s) will reside later.

The `ngramList` variable is another empty list and is where your resulting ngrams will reside. 

In [4]:
data = "*.txt"
nltkStop = True
customStop = True
ng = 2
stopLang = 'english'
stopWords = []
cleanText = []
ngramList = []

#print(" ".join(stopwords.fileids()))

### Stopwords
If you set `nltkStop` equal to **True** above then this will add the nltk stopwords list to the empty list named `stopWords`.

If you wish to add additional words to the stopWords list then add the word in quotes to the list in `stopWords.extend(['the', 'words', 'you', 'want', 'to', 'add'])`.

In [5]:
# NLTK Stop words
if nltkStop is True:
    stopWords.extend(stopwords.words(stopLang))

    stopWords.extend(['would', 'said', 'says', 'also'])

#### Add own stopword list

Here is where your own stopwords list is added if you selected **True** in `customStop` above. Here you will need to change the folder names and file name to match your folders and file. Remember to put each folder name in quotes and in the correct order always putting the file name including the file extension (.txt) last.

In [6]:
if customStop is True:
    stopWordsFilepath = os.path.join(homePath, "Text-Analysis-DavidBranchV2", "data", "earlyModernStopword.txt")

    with open(stopWordsFilepath, "r",encoding = 'utf-8') as stopfile:
        stopWordsCustom = [x.strip() for x in stopfile.readlines()]

    stopWords.extend(stopWordsCustom)

### Functions
We need to create a function in order to calculate and create a wordcloud. Any time you see `def` that means we are DEclaring a Function. The `def` is usually followed by the name of the function being created and then in parentheses are the parameters required by the function. After the parentheses is a colon, which closes the declaration, then a bunch of code below which is indented. The indented code is the program statement or statements to be executed. Once you have created your function all you need to do in order to run it is call the function by name and make sure you have included all the required parameters in the parentheses. This allows you to call the function without having to write out all the code in the function every time you wish to perform that task.

#### Text Cleaning

This function does some text cleaning for us and requires the parameter: text (as in what text are you cleaning).

Now we come to the statements to be executed. First we lowercase the text or else 'Love' and 'love' will be counted as two different words, so we make them all 'love'. Then we split the text into individual words and remove empty spaces. Then we remove any digits, stopwords, and punctuation and return a list of cleaned words.

In [7]:
def textClean(text):
    
    text = text.strip().lower()
    
    tokens = re.split(r'\W+', text )
    
    # remove empty string
    tokens = [t for t in tokens if t]
    
    # remove digits
    tokens = [t for t in tokens if not t.isdigit()]
    
    # built-in stop words list
    tokens = [t for t in tokens if t not in stopWords]
        
    # remove punctuation
    puncts = list(string.punctuation)
    puncts.append('--')

    tokens = [t for t in tokens if t not in puncts]

    return tokens

#### Reading in the Text

Now we read in the text. You chose earlier whether to read in a single text or the entire directory, so there should be no need to make changes here.

In [8]:
for path in glob.glob(os.path.join(dataHome, data)):
    with open(path, "r") as file:
         # skip hidden file
        if path.startswith('.'):
            continue
        text = file.read()
        cleanText.extend(textClean(text))

No changes needed here. This just converts our text to a str object so we can find ngrams later.

In [9]:
cleanTokens = ' '.join(cleanText)

### Find Ngrams

We use the textblob package to get ngrams. We use the `ng` variable we created earlier to determine if we are interested in unigrams, bigrams, or trigrams. There should be no reason to make changes here.

In [10]:
blob = TextBlob(cleanTokens)

if ng == 1: 
    nGrams = blob.ngrams(n=1)
if ng == 2:
    nGrams = blob.ngrams(n=2)
if ng == 3:
    nGrams = blob.ngrams(n=3)

Now we are converting our ngrams to a list which we can then put into a dataframe to be turned into a wordcloud.

In [11]:
for wlist in nGrams:
   ngramList.append(' '.join(wlist))

Now we make our dataframe. You won't need to make changes to this code. Just note that we had to go through the dataframe and replace the space between our ngrams with an underscore. This is because the Python wordcloud package has trouble handling the space, so we connect our ngrams with an underscore so that the wordcloud package will see it as one word, but humans can see it as two or three, depending on if you chose bigrams or trigrams above.

In [12]:
df = pd.DataFrame(ngramList)
df = df.replace(' ', '_', regex=True)
dfCounts = df[0].value_counts()
countsDF = pd.DataFrame(dfCounts)
countsDF.reset_index(inplace = True)
df_C = countsDF.rename(columns={'index':'ngrams',0:'freq'})
df_C.set_index(df_C['ngrams'], inplace = True)
df_C['ngrams'] = df_C['ngrams'].astype(str)
dfNG = df_C.sort_values('freq', ascending = False)

Now lets see what our dataframe looks like. If you want to see more, just change the number in parentheses.

In [13]:
dfNG.head(10)

,ngrams,freq
ngrams,,
good_lord,good_lord,243
come_come,come_come,157
come_hither,come_hither,112
good_morrow,good_morrow,109
good_night,good_night,83
come_let,come_let,70
lord_lord,lord_lord,64
love_love,love_love,64
old_man,old_man,63


### Plot our wordcloud

We begin by determining if we want to use a mask to shape the wordcloud. The wordcloud package uses an image file which is refered to as a mask to determine the shape. The resulting wordcloud will be in the shape of the chosen image. If we want to use a mask we need to set `useMask` equal to **True**. 

We have images to use for making wordcloud shapes and the path in the first line points to the folder the images are stored in. Simply adjust the path to point to where this folder is located on Carbonate for you. Then in the next line choose the name of the image file you wish to use. It is best to choose an image with a lot of contrast between the image and the background, such as a stencil where the image is all black and the background is white.

Next we choose the maximum number of ngrams we want in our wordcloud by assigning the number to the variable `maxWrdCnt`.

Then we choose a background color and assign it the variable `bgColor`. 

Next we choose the color of the words in the wordcloud and assign it to the variable `color`. The current color is from the RColorBrewer palette of colors. You can find other color options [here](https://www.nceas.ucsb.edu/~frazier/RSpatialGuides/colorPaletteCheatsheet.pdf). Just put the name of the color selection you want in quotes.

Now we choose the figure size. The first number is the width and the second number is the height. Feel free to make changes as needed. We assign the figure size to the variable `FigureSz`.

Then we name the output '.png' file our wordcloud will be saved as and assign it to the variable `wcOutputFile`. 

Next we determine the format of the output file. We already named our file above with a '.png' file type. So we need to make sure this matches that file type. We assign the format to the variable `imgFmt`.

Then we choose the resolution of our output image by assigning the dpi resolution we want to the variable `dpi`.

Then we create an additional list of stopwords that will remove problematic ngrams. Just remember to type the ngram with an underscore between the two words. Then we remove the row containing the ngram from the database. This keeps you from having to go up and remove single words (and potentially other interesting ngrams) as you can now just remove the specific ngram.

Next we give parameters for our wordcloud. and save them as `wc`. Here we have an 'if else' statement that runs one line if we set `useMask` equal to **True** or it will run the other (`else:`) if it is equal to **False**. Any changes to these lines of code were already decided in the variable at the beginning of the cell.

Then are the statements for how the wordcloud is displayed. We already chose the figure size above, so we can leave `plt.figure(figsize = figureSz)` alone.  Then we want it to look like what we described in our `wc` variable which is what the `plt.imshow(wc, interpolation = 'bilinear')` statement does. Then we need to state that we are not using an x or y axis by using the `plt.axis("off")` staement. Next we want the layout to be tight instead of spread out so we use the `plt.tight_layout()` statement. Then we state arguments for how we want the wordcloud saved to file (`plt.savefig(os.path.join(dataResults, wcOutputFile), format = imgFmt, dpi = dpi, bbox_inches = 'tight')`), and finally that we want to see the final result displayed in the notebook which is what`plt.show()` does.

Run the code and generate your ngram wordcloud!!

In [14]:
# Variables
useMask = False
maskPath = os.path.join(homePath, 'Text-Analysis-master','data','wordcloudMasks')
mask = np.array(Image.open(os.path.join(maskPath, "Shakespeare.png")))
maxWrdCnt = 500
bgColor = "black"
color = "Dark2"
figureSz = (80,40)
wcOutputFile = "ngramWordCloud.png"
imgFmt = "png"
dpi = 300

# Ngram Stopwords
stopwords = ["ngrams","good_lord","come_come"]
text = dfNG[~dfNG['ngrams'].isin(stopwords)]

# Wordcloud aesthetics
#if useMask is True:    
    #wc = wordcloud.WordCloud(background_color = bgColor, max_words = maxWrdCnt, colormap = color, mask = mask).generate_from_frequencies(text['freq'])
#else:
    #wc = wordcloud.WordCloud(background_color = bgColor, max_words = maxWrdCnt, colormap = color, mask = None).generate_from_frequencies(text['freq'])

# show
#plt.figure(figsize = figureSz)
#plt.imshow(wc, interpolation = 'bilinear')
#plt.axis("off")
#plt.tight_layout()
    
# save graph as an image to file
#plt.savefig(os.path.join(dataResults, wcOutputFile), format = imgFmt, dpi = dpi, bbox_inches = 'tight')
    
#plt.show()

## VOILA!!